# Basic Simulation Example

This notebook demonstrates:
1. Loading a pre-generated network
2. Running a basic simulation
3. Visualizing results with Altair

## Setup and Imports

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import networkx as nx

import variant_dynamics as vd
from variant_dynamics.network_construction.load_example_networks import load_ba32
from variant_dynamics.simulation.full_system.one_layer_gillespie import (
    simulation_phylogeny,
)
from variant_dynamics.utility.plot_altair import altair_helvetica_theme

In [2]:
# Configure Altair theme
# switch between "light" and "dark" for better visibility in light/dark mode
altair_helvetica_theme("dark")

## 1. Load a Pre-Generated Network

Load a small Barabási-Albert network with 32 nodes

In [3]:
# Load the network
A = load_ba32()

In [4]:
# Compute eigenvector centrality for visualization
G_networkA = nx.from_pandas_adjacency(A)
centrality = nx.eigenvector_centrality(G_networkA)

## 2. Run Simulation

In [5]:
# Simulation parameters
mu = 0.1  # Mutation rate
T = 200.0 / (mu * 32)  # Total simulation time
s_mean = 0.1  # fitness increase per mutation
rng_seed = 42  # Random seed for reproducibility

# Run the simulation
phylogeny = simulation_phylogeny(
    T=T,
    A=A.values,
    mu=mu,
    fitness_increase="constant",
    s_mean=s_mean,
    rng_seed=rng_seed,
)

print("The first rows of the phylogeny data generated by the simulation:")
phylogeny.head()

The first rows of the phylogeny data generated by the simulation:


,fitness,origin,t,predecessor,extinction,fixation,survival
0,1.0,NaN,0.000000,NaN,NaN,True,True
1,1.1,13.0,0.080075,0.0,NaN,False,False
2,1.1,27.0,0.180198,0.0,NaN,False,False
3,1.1,16.0,0.652815,0.0,NaN,False,False
4,1.1,11.0,0.668660,0.0,NaN,True,True


## 3. Visualize the Results

Plot the fitness of new clones over time. 

In [6]:
def plot_strain_evolution_nwA(phylogeny, centrality=centrality):

    df_strain_evolution = (
        phylogeny.query("predecessor.notna()")
        .assign(
            predecessor=lambda df: df.predecessor.astype(int),
            origin=lambda df: df.origin.astype(int),
        )
        .pipe(
            lambda df: pd.concat(
                [
                    df.assign(original=True),
                    df.loc[:, ["predecessor"]]
                    .join(
                        phylogeny.loc[:, ["origin", "fitness", "t"]].assign(
                            origin=lambda df: df.origin.fillna(0).astype(int),
                        ),
                        on="predecessor",
                    )
                    .assign(original=False),
                ]
            )
        )
        .assign(eigenv_centrality=lambda df: df.origin.map(centrality))
        .reset_index()
        .rename(columns={"index": "strain"})
        .drop(columns=["predecessor"])
    )
    lineage_plot = (
        alt.Chart(df_strain_evolution)
        .encode(
            x="t",
            y=alt.Y("fitness:Q"),
            detail=alt.Color("strain:N"),
        )
        .mark_line(stroke="#888")
        .properties(
            width=400,
            height=400,
            title=alt.TitleParams("Strain Evolution", anchor="start"),
        )
    )
    strain_plot = (
        alt.Chart(
            df_strain_evolution.query("original").sort_values(
                ["t", "eigenv_centrality"]
            )
        )
        .encode(
            x="t",
            y=alt.Y("fitness:Q"),
            fill=alt.Color("eigenv_centrality:Q", scale=alt.Scale(scheme="greens")),
            detail=alt.Color("strain:N"),
        )
        .mark_circle(opacity=1)
        .properties(
            width=400,
            height=400,
            title=alt.TitleParams("Strain Evolution", anchor="start"),
        )
    )

    return lineage_plot + strain_plot


plot_strain_evolution_nwA(phylogeny)

alt.LayerChart(...)